In [1]:
import os
from langchain.vectorstores import FAISS
from langchain.docstore import InMemoryDocstore
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.schema import Document
import faiss



In [2]:
os.path.exists("conda_env_test.ipynb")

True

In [3]:

os.chdir('..\..')

In [4]:
from app.models.bart_large_cnn import BartSummaryModel
Summary_Model = BartSummaryModel()

c:\Users\nanfangwuyu\.conda\envs\novel_rag\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Device set to use cuda:0


In [5]:
text = """
**Chapter 1: Embers of Destiny**

In the realm of Eldoria, where the five elements danced in a delicate balance, the air crackled with a potential that whispered of ancient magics. Fire, fierce and unyielding, flickered in the heart of the volcanoes; water flowed with grace through the rivers and oceans, nurturing life in its embrace; wind roamed free, a playful spirit that caressed the mountains; electricity surged in the stormy skies, a wild tempest of power; and earth, the steadfast guardian, cradled the very foundations of existence. Here, in this vibrant tapestry woven by elemental forces, the fate of kingdoms hung by a thread.

Amidst the verdant valleys of Eldoria stood Jack, a young mage with a spirit as untamed as the tempest itself. He was a figure of quiet strength, with tousled chestnut hair that caught the light of the setting sun and eyes as deep as the ocean’s depths—reflecting a soul that yearned for adventure. Jack had long been aware of his gifts, a rare affinity for all five elements that set him apart from others. Yet, despite his burgeoning power, he felt the weight of an uncertain destiny resting heavily on his shoulders.

As he stood atop the cliffs of Varyn, the wind tousling his hair, Jack gazed out at the swirling mists of the horizon. The clouds, tinged with the colors of twilight, whispered promises of journeys yet to be taken. Today, a flicker of intuition ignited within him, a stirring that foretold an awakening—a call to harness the elemental forces that lay dormant, waiting for the right moment to erupt. With a heart full of determination and a mind sharpened by dreams, Jack took a deep breath, ready to carve his path through the ever-shifting tides of fate. Little did he know, the world was on the brink of change, and he was destined to be its catalyst.
"""
Summary_Model.summarize(text)

'Jack is a young mage with a spirit as untamed as the tempest itself. He has a rare affinity for all five elements that set him apart from others. Despite his burgeoning power, he felt the weight of an uncertain destiny resting heavily on his shoulders. Little did he know, the world was on the brink of change.'

In [4]:

from app.managers import vector_manager as vm
import os

SAVE_DIR = "data/samples/raws"
os.makedirs(SAVE_DIR, exist_ok=True)



def get_latest_chapter_num(store_type="summary"):
    vectorstore = vm.load_vectorstore(store_type)
    all_docs = vectorstore.docstore._dict.values()
    chapter_numbers = [doc.metadata.get("chapter", 0) for doc in all_docs if isinstance(doc.metadata.get("chapter", 0), int)]
    latest_chapter_num = max(chapter_numbers) if chapter_numbers else 0
    return latest_chapter_num

get_latest_chapter_num()

c:\Users\nanfangwuyu\.conda\envs\novel_rag\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Device set to use cuda:0


Reload existing faiss


0

In [4]:
def create_new_vectorstore(embedding_model):
    index = faiss.IndexFlatL2(len(embedding_model.embed_query("hello world")))
    vectorstore = vector_store = FAISS(
        embedding_function=embedding_model,
        index=index,
        docstore=InMemoryDocstore(),
        index_to_docstore_id={},
    )
    return vectorstore

In [5]:
VECTORSTORE_PATH = "data/vectorstore/summary_index"
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Step 0: 加载或初始化向量库
if os.path.exists(VECTORSTORE_PATH):
    print(True)
    vectorstore = FAISS.load_local(VECTORSTORE_PATH, embedding_model, allow_dangerous_deserialization=True)
else:
    vectorstore = create_new_vectorstore(embedding_model)
    

True


In [6]:
vectorstore = create_new_vectorstore(embedding_model)
vectorstore.docstore._dict

{}

In [28]:
# Step 1: 获取所有文档 & 最新章节编号
all_docs = vectorstore.docstore._dict.values()
chapter_numbers = [doc.metadata.get("chapter", 0) for doc in all_docs if isinstance(doc.metadata.get("chapter", 0), int)]
latest_chapter = max(chapter_numbers) if chapter_numbers else 0


In [29]:
all_docs, latest_chapter

(dict_values([Document(id='e948cd24-4678-436f-b98f-9514db311f42', metadata={'chapter': 1}, page_content='In the magical realm of Eldoria, where fire, water, wind, electricity, and earth govern existence, Jack, a unique mage cloaked in indigo, possesses the rare ability to master all five elements. Revered and envied by townsfolk, he grapples with the loneliness of his gifts. As he stands at the edge of the Whispering Woods one evening, feeling the elemental energies around him, he senses a change in the air. Unbeknownst to him, the universe is weaving a fate that will challenge the core of his powers and alter his destiny.'), Document(id='27fff7cd-20be-4b44-be42-50af318b6e80', metadata={'chapter': 2}, page_content='In the twilight of Eldoria, Jack stands at the edge of the Whispering Woods, feeling a deep connection to the ancient trees. Drawn by an electric anticipation, he ventures into the forest, where reality blurs and whispers grow louder. Amidst the scents of earth and nightshad

In [30]:
# Step 2: 获取最新章节 summary
latest_summary_docs = [doc for doc in all_docs if doc.metadata.get("chapter") == latest_chapter]
latest_summary_doc = latest_summary_docs[0] if latest_summary_docs else ""



In [31]:
# Step 3: 用户主题输入
if latest_chapter == 0:
    user_query = "Help me write a fantasy novel. Introduces the worldview and the male protagonist. The world has five elements: fire, water, wind, electricity, and earth. The male protagonist names Jack, is a powerful mage." 
else:
    user_query = "Continue writing."

# 检索相关章节概要
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})
related_docs = retriever.get_relevant_documents(user_query)
# related_chapter_numbers = [doc.metadata.get("chapter", 0) for doc in related_docs if isinstance(doc.metadata.get("chapter", 0), int)]
if latest_summary_doc in related_docs:
    related_docs.remove(latest_summary_doc)
related_summaries = "\n\n".join(["Summary of Chapter {}:\n".format(doc.metadata.get("chapter")) + doc.page_content for doc in related_docs])

In [32]:
related_docs, related_summaries

([Document(id='e948cd24-4678-436f-b98f-9514db311f42', metadata={'chapter': 1}, page_content='In the magical realm of Eldoria, where fire, water, wind, electricity, and earth govern existence, Jack, a unique mage cloaked in indigo, possesses the rare ability to master all five elements. Revered and envied by townsfolk, he grapples with the loneliness of his gifts. As he stands at the edge of the Whispering Woods one evening, feeling the elemental energies around him, he senses a change in the air. Unbeknownst to him, the universe is weaving a fate that will challenge the core of his powers and alter his destiny.')],
 'Summary of Chapter 1:\nIn the magical realm of Eldoria, where fire, water, wind, electricity, and earth govern existence, Jack, a unique mage cloaked in indigo, possesses the rare ability to master all five elements. Revered and envied by townsfolk, he grapples with the loneliness of his gifts. As he stands at the edge of the Whispering Woods one evening, feeling the eleme

In [33]:
# Step 4: 构建 prompt 生成内容
if latest_chapter > 1:

    prompt = f"""You are a novelist working on a long novel.

Here is a summary of the previously on few related chapters:

"{related_summaries}"

And here is a summary of the previously on the last chapter (Chapter {latest_summary_doc.metadata.get("chapter")}): 

"{latest_summary_doc.page_content}"

Please continue to WRITE the CONTENT of the NEXT CHAPTER - chapter {latest_chapter + 1}, keeping it logically coherent, within about 300 words. 

Here is what the user expects to see in the next chapter: "{user_query}"

    """
elif latest_chapter == 1:
    prompt = f"""You are a novelist working on a long novel.

Here is a summary of the previously on the first chapter (Chapter {latest_summary_doc.metadata.get("chapter")}): 

"{latest_summary_doc.page_content}"

Please continue to WRITE the CONTENT of the NEXT CHAPTER - chapter {latest_chapter + 1}, keeping it logically coherent, within about 300 words. 

Here is what the user expects to see in the next chapter: "{user_query}"
"""
else:
    prompt = f"""You are a novelist working on a long novel.

Please WRITE the content of the chapter 1 within about 300 words. 

Here is what the user expects to see in the chapter 1: "{user_query}"

    """

#   # 你封装的模型类，含 generate & summarize 方法

print(prompt)

You are a novelist working on a long novel.

Here is a summary of the previously on few related chapters:

"Summary of Chapter 1:
In the magical realm of Eldoria, where fire, water, wind, electricity, and earth govern existence, Jack, a unique mage cloaked in indigo, possesses the rare ability to master all five elements. Revered and envied by townsfolk, he grapples with the loneliness of his gifts. As he stands at the edge of the Whispering Woods one evening, feeling the elemental energies around him, he senses a change in the air. Unbeknownst to him, the universe is weaving a fate that will challenge the core of his powers and alter his destiny."

And here is a summary of the previously on the last chapter (Chapter 2): 

"In the twilight of Eldoria, Jack stands at the edge of the Whispering Woods, feeling a deep connection to the ancient trees. Drawn by an electric anticipation, he ventures into the forest, where reality blurs and whispers grow louder. Amidst the scents of earth and 

In [ ]:
from app.models.model import LLM
generated_chapter = LLM.generate(prompt, max_tokens=512)
generated_summary = LLM.summarize(generated_chapter)

In [35]:
print(f"Generated Chapter:\n\n{generated_chapter} \n\nGenerated Summary:\n\n{generated_summary}")

Generated Chapter:

**Chapter 3: The Awakening of Shadows**

Jack stepped into the clearing, the air crackling with an otherworldly energy that hummed beneath his skin. The stone pedestal, a relic of ages past, loomed before him, its surface etched with runes that glowed faintly in the twilight. The whispers of the forest grew louder, transforming into a symphony of voices, each one tugging at the corners of his mind, urging him to unlock the secrets hidden within the stone.

As he approached, the flickering glow intensified, casting dancing shadows across the trees. Jack could feel the weight of history pressing upon him, as if the very essence of the forest had gathered to witness this moment. A sense of purpose surged within him, igniting a flicker of hope amidst the loneliness he had often felt. Was this the connection he had long sought—the intertwining of his fate with something greater than himself?

Tentatively, he reached out, his fingers brushing against the cool stone. Insta

In [ ]:
# generated_summary = "Jack and Jill went up the hill to fetch a pail of water. Jack fell down and broke his crown, and Jill came tumbling after."  # 👈 替换成模型生成的内容
# generated_summary = "Lucy and her brother went to the market to buy some groceries. They met a friendly dog on the way."  # 👈 替换成模型生成的内容
# generated_summary = "The sun was shining brightly as they walked home. They had a great time at the market."  # 👈 替换成模型生成的内容

In [36]:
# Step 5: 构建并添加文档
latest_chapter += 1
new_doc = Document(
    page_content=generated_summary,
    metadata={"chapter": latest_chapter}
)
vectorstore.add_documents([new_doc])


['a855b3b5-75ab-4e3b-88c7-81763ee7fd7f']

In [37]:
vectorstore.docstore._dict

{'e948cd24-4678-436f-b98f-9514db311f42': Document(id='e948cd24-4678-436f-b98f-9514db311f42', metadata={'chapter': 1}, page_content='In the magical realm of Eldoria, where fire, water, wind, electricity, and earth govern existence, Jack, a unique mage cloaked in indigo, possesses the rare ability to master all five elements. Revered and envied by townsfolk, he grapples with the loneliness of his gifts. As he stands at the edge of the Whispering Woods one evening, feeling the elemental energies around him, he senses a change in the air. Unbeknownst to him, the universe is weaving a fate that will challenge the core of his powers and alter his destiny.'),
 '27fff7cd-20be-4b44-be42-50af318b6e80': Document(id='27fff7cd-20be-4b44-be42-50af318b6e80', metadata={'chapter': 2}, page_content='In the twilight of Eldoria, Jack stands at the edge of the Whispering Woods, feeling a deep connection to the ancient trees. Drawn by an electric anticipation, he ventures into the forest, where reality blur

In [20]:
# Step 6: 保存向量库
vectorstore.save_local(VECTORSTORE_PATH)

# Step 7: 保存章节内容和总结到本地（你已有逻辑）
# save_chapter_text(latest_chapter + 1, generated_chapter)
# save_summary_text(latest_chapter + 1, generated_summary)
